In [7]:
import pandas as pd
import hashlib
import os
import pyodbc
import warnings
warnings.filterwarnings("ignore")

In [8]:
# Funciones para anonimizar
def generate_salt():
    """
    Genera una cadena aleatoria (salt) para añadir a los datos antes de hashearlos.
    """
    return os.urandom(16)  # 16 bytes para el salt (128 bits)

def hash_with_salt(data, salt):
    """
    Aplica el algoritmo SHA-512 para hashear los datos junto con el salt.
    """
    data_with_salt = data.encode() + salt
    hashed_data = hashlib.sha512(data_with_salt).hexdigest()
    return hashed_data

# Funciones para interactuar con la db
def get_table_names(conn_str):
    """
    Obtiene una lista de nombres de tablas disponibles en la base de datos.
    """
    with pyodbc.connect(conn_str) as conn:
        cursor = conn.cursor()
        tables = [table.table_name for table in cursor.tables(tableType='TABLE')]
        return tables

def execute_query(conn_str, query):
    """
    Ejecuta una consulta SQL en la base de datos y devuelve los resultados en un DataFrame de pandas.
    """
    with pyodbc.connect(conn_str) as conn:
        df = pd.read_sql_query(query, conn)
    return df

In [10]:
# Conexion a la db
conn_str = r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=C:\Users\CityLab Biobio - DS\Desktop\Citylab\datos\Bases de datos nuevas\Sectra\Sectra _2017 _CCP\BD\07CLBB_EOD_del_GRAN_CCP.accdb;'

In [19]:
## Obtener los nombres de las tablas disponibles en la db
# table_names = get_table_names(conn_str)
# print("Tablas disponibles:")
# for name in table_names:
#     print(name)

In [4]:
# Nombre de las tablas y columnas a anonimizar
anon_columns = {
    'HOGAR': ['DireccionTelefono1', 'DireccionTelefono2', 'Encuestador',],
    'PERSONA': ['TelefonoContacto1', 'TelefonoContacto2', 'Encuestador', 'Nombre', ],
}

# Tablas que no requieren anonimizar
direct_tables = ['VIAJES_LAB', 'Tipo Asiento']

# Ruta principal para guardar los datos de salida
save_path = 'C:\\Users\\CityLab Biobio - DS\\Dev\\CityLab_Biobio\\data\\output'

In [16]:

# Iteramos en las tablas con sus respectivas columnas
for table, anon_cols in anon_columns.items():
    # Query para consultar todas las columnas de la tabla
    query = f'SELECT * FROM {table}'
    
    # Obtener los datos desde la db
    df = execute_query(conn_str, query)

    # Generamos un Dataframe para guardar los datos con y sin anonimizacion
    df_hash = df[anon_cols]

    # Iteramos en las columnas a anonimizar
    for col in anon_cols:
        # Generar un nuevo salt para la columna correspondiente
        salt = generate_salt()

        # Nuevo nombre de la columna a anonimizar
        new_col = f'{col}_anon'

        # Aplicar hash y salt a cada valor de la columna correspondiente
        new_data = df[col].apply(lambda x: hash_with_salt(x, salt))

        # Agregamos los datos a los Dataframe
        df[new_col] = new_data
        df_hash[new_col] = new_data

    # Quitamos las columnas que se anonimizaron
    df.drop(columns=anon_cols, inplace=True)

    # Guardamos los dataframe resultantes
    df.to_pickle(os.path.join(save_path, 'transfer', f'{table}.pkl'))
    df_hash.to_pickle(os.path.join(save_path, 'hash', f'{table}.pkl'))

In [18]:
# Exportar las tablas que no requieren anonimización
for table in direct_tables:
    # Query para consultar todas las columnas de la tabla
    if(' ' in table):
        query = f"SELECT * FROM [{table}];"
    else:
        query = f"SELECT * FROM {table};"
    
    # Obtener los datos desde la db
    df = execute_query(conn_str, query)

    # Generamos un Dataframe para guardar los datos con y sin anonimizacion

    df.to_pickle(os.path.join(save_path, 'transfer', f'{table}.pkl'))